In [40]:
import pandas as pd
from transformers import *
import fasttext
import fasttext.util

In [3]:
all_features = ["tweet_id" , "tweet_timestamp"]
train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [2, 8], sep="\x01"
                   )
print('train loaded')

train loaded


In [4]:
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
print('train sorted')
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100)))

train sorted


In [5]:
train_set_embeddings = pd.read_csv('s3://recsys-challenge-2020/all_lang_output_train.csv')

In [6]:
val_set_embeddings = pd.read_csv('s3://recsys-challenge-2020/all_lang_output_val.csv')

In [7]:
train_embeddings_set = pd.concat([train_set, train_set_embeddings], axis = 1)
val_embeddings_set = pd.concat([val_set.reset_index(), val_set_embeddings], axis =1)
train_embeddings = pd.concat([train_embeddings_set, val_embeddings_set])

In [8]:
train_embeddings_index_sorted = train_embeddings.sort_values(['index'])

In [15]:
desired_line = pd.read_csv('s3://recsys-challenge-2020/training.tsv', skiprows = 123786896,\
                           header = None, sep = '\x01', nrows = 1)

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [42]:
desired_text = tokenizer.decode(list(map(int, desired_line[0][0].split('\t'))))

In [28]:
desired_line[7][0]

'125C57F4FA6D4E110983FB11B52EFD4E'

In [31]:
language_name_code_df = pd.read_csv('s3://recsys-challenge-2020/language_code_name_pairs.csv')

In [35]:
language_name_code_df[language_name_code_df.code == '125C57F4FA6D4E110983FB11B52EFD4E']

,code,name
4,125C57F4FA6D4E110983FB11B52EFD4E,ko


In [ ]:
fasttext.util.download_model('ko', if_exists='ignore')  # Korean
ft = fasttext.load_model('cc.ko.300.bin')
fasttext.util.reduce_model(ft, 20)

In [64]:
desired_embedding = ft.get_sentence_vector(desired_text)

In [45]:
train_embeddings_index_sorted.set_index('index', inplace = True)

In [91]:
train_ft_embeddings = train_embeddings_index_sorted[['sentence_embedding', 'tweet_id']]

In [92]:
train_ft_embeddings.at[123786896, 'sentence_embedding'] = str(desired_embedding)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3041: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc[index, col] = value


In [93]:
train_ft_embeddings.sort_index(inplace = True)

### Check if train_ft_embeddings are sorted in the same way as original train

In [103]:
train_ft_embeddings[['tweet_id']].equals(train[['tweet_id']])

False

In [116]:
ne = (train[['tweet_id']] != train_ft_embeddings[['tweet_id']]).any(1)

In [119]:
train[ne]

,tweet_id,tweet_timestamp
123786896,F1529D503347DA75EFFD67CC1D9DA0F9,1581486678


In [121]:
train_ft_embeddings[ne]

,sentence_embedding,tweet_id
123786896,[-0.41131395 -0.3181513 0.26452422 0.026240...,NaN


In [100]:
train_ft_embeddings.index.name = None

In [122]:
train_ft_embeddings[['sentence_embedding']].to_csv('s3://recsys-challenge-2020/all_lang_output_train.csv')